In [1]:
from warnings import filterwarnings
import sys
filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import tensorflow as tf

%matplotlib inline

from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


#from tqdm import tqdm_notebook
from notifyme import notify
from gc import collect
from time import time,sleep
from os import path,system
from json import dumps

In [2]:
labels = pd.read_csv("./train.csv")

In [3]:
labels.nunique()

image_id               200840
grapheme_root             168
vowel_diacritic            11
consonant_diacritic         7
grapheme                 1295
dtype: int64

In [4]:
grapheme_root_ohe = OneHotEncoder(dtype=np.uint16,sparse=False)
vowel_diacritic_ohe = OneHotEncoder(dtype=np.uint16,sparse=False)
consonant_diacritic_ohe = OneHotEncoder(dtype=np.uint16,sparse=False)

grapheme_root_ohe.fit(labels[['grapheme_root']])
vowel_diacritic_ohe.fit(labels[['vowel_diacritic']])
consonant_diacritic_ohe.fit(labels[['consonant_diacritic']])

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.uint16'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [5]:
# inputs = Input(shape = (48, 48, 1),name="inputs")
# model = Conv2D(filters=32, kernel_size=(4, 4), padding='SAME', activation='relu', input_shape=(48,48,1))(inputs)
# model = Conv2D(filters=32, kernel_size=(4, 4), padding='SAME', activation='relu')(model)
# model = BatchNormalization(momentum=0.15)(model)
# model = MaxPool2D(pool_size=(2, 2))(model)
# model = Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
# model = Dropout(rate=0.3)(model)

# model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
# model = Conv2D(filters=64, kernel_size=(4, 4), padding='SAME', activation='relu')(model)
# model = BatchNormalization(momentum=0.15)(model)
# model = MaxPool2D(pool_size=(2, 2))(model)
# model = Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
# model = BatchNormalization(momentum=0.15)(model)
# model = Dropout(rate=0.3)(model)

# model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
# model = Conv2D(filters=128, kernel_size=(4, 4), padding='SAME', activation='relu')(model)
# model = BatchNormalization(momentum=0.15)(model)
# model = MaxPool2D(pool_size=(2, 2))(model)
# model = Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
# model = BatchNormalization(momentum=0.15)(model)
# model = Dropout(rate=0.3)(model)

# model = Conv2D(filters=256, kernel_size=(6, 6), padding='SAME', activation='relu')(model)
# model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
# model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
# model = BatchNormalization(momentum=0.15)(model)
# model = MaxPool2D(pool_size=(2, 2))(model)
# model = Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
# model = BatchNormalization(momentum=0.15)(model)
# model = Dropout(rate=0.3)(model)

# model = Flatten()(model)
# model = Dense(1024, activation = "relu")(model)
# model = Dropout(rate=0.3)(model)
# dense = Dense(512, activation = "relu")(model)

# head_root = Dense(168, activation = 'softmax',name="grapheme_root")(dense)
# head_vowel = Dense(11, activation = 'softmax',name='vowel_diacritic')(dense)
# head_consonant = Dense(7, activation = 'softmax',name='consonant_diacritic')(dense)

# model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

In [6]:
inputs = Input(shape = (48, 48, 1),name="inputs")
model = Conv2D(filters=32, kernel_size=(8, 8), padding='SAME', activation='relu', input_shape=(48,48,1))(inputs)
model = Conv2D(filters=32, kernel_size=(6, 6), padding='SAME', activation='relu')(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=64, kernel_size=(4, 4), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(4, 4), padding='SAME', activation='relu')(model)

model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Flatten()(model)
model = Dense(4096, activation = "relu")(model)
model = Dropout(rate=0.3)(model)
dense = Dense(1024, activation = "relu")(model)

head_root = Dense(168, activation = 'softmax',name="grapheme_root")(dense)
head_vowel = Dense(11, activation = 'softmax',name='vowel_diacritic')(dense)
head_consonant = Dense(7, activation = 'softmax',name='consonant_diacritic')(dense)

outputs = [
        head_root, 
#         head_vowel, 
#         head_consonant
]

model = Model(inputs=inputs, outputs=outputs)

In [7]:
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:

def crop(img,pad=True):
    W_THRESH = 8
    H_THRESH = 8
    PAD = 3 if pad else 0

    W_MIN,W_MAX = np.where(img.std(axis=0) > W_THRESH)[0][[0,-1]]
    H_MIN,H_MAX = np.where(img.std(axis=1) > H_THRESH)[0][[0,-1]]
    
    return np.pad(img[H_MIN:H_MAX,W_MIN:W_MAX],PAD,constant_values=253)

def resize(img):
    img = crop(img.reshape(137,236).astype(np.uint8))
    ret,img = cv.threshold(img,110,255,cv.THRESH_BINARY_INV)    
    return cv.resize(img,(48,48)).astype(np.uint8).reshape(48,48,1)

def input_flow(x,y,batch_size=200,epochs=1):
    for epoch in range(epochs):
        for i in range(batch_size,x.shape[0],batch_size):
            rows = x.iloc[i-batch_size:i].values
            yield (
                    {
                        "inputs":np.apply_along_axis(resize,axis=1,arr=rows)/255
                    },
                    {
                        "grapheme_root":y[0][i-batch_size:i],
                        'vowel_diacritic':y[1][i-batch_size:i],
                        'consonant_diacritic':y[2][i-batch_size:i]
                    }
                )
        rows = x.iloc[i:].values
        yield (
                    {
                        "inputs":np.apply_along_axis(resize,axis=1,arr=rows)/255
                    },
                    {
                        "grapheme_root":y[0][i:],
                        'vowel_diacritic':y[1][i:],
                        'consonant_diacritic':y[2][i:]
                    }
                )

In [9]:
def get_train_test(file_id):
    df = pd.merge(
            pd.read_parquet(f"./train_image_data_{file_id}.parquet"),
            labels,
            on='image_id'
        )
    
    grapheme_root = grapheme_root_ohe.transform(df.grapheme_root.values.reshape(-1,1))
    vowel_diacritic = vowel_diacritic_ohe.transform(df.vowel_diacritic.values.reshape(-1,1))
    consonant_diacritic = consonant_diacritic_ohe.transform(df.consonant_diacritic.values.reshape(-1,1))
    
    df = df.drop(columns=['image_id','grapheme_root','vowel_diacritic','consonant_diacritic','grapheme'])
    
    return df,(grapheme_root,vowel_diacritic,consonant_diacritic)

In [10]:
checkpoint_path = ".keras_checkpoints/chek.ckpt"
checkpoint_dir = path.dirname(checkpoint_path)

cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

BATCH_SIZE = 400
EPOCHS = 10
SUB_EPOCHS = 2
START_EPOCH = 0

HALT = True

# if RESUME:
#     log = open("./log.txt",'r').read().split(" ")
#     START_EPOCH = int(log[2])

#     latest = tf.train.latest_checkpoint("./.keras_checkpoints")
#     model.load_weights(latest)

In [11]:
for epoch in range(START_EPOCH,EPOCHS):
    hist = None
    for file_id in range(4):
        print (f"EPOCH : {epoch} | FILE : {file_id}")
        X,Y = get_train_test(file_id)
        gen = input_flow(X,Y,batch_size=BATCH_SIZE,epochs=SUB_EPOCHS)
        
        model.fit_generator(gen,steps_per_epoch=X.shape[0]//BATCH_SIZE,epochs=SUB_EPOCHS)

        del X,Y
        collect()
        if HALT:
            sleep(90)
    
        open("./log.txt","w+").write(f"EPOCH : {epoch} | FILE : {file_id}")

    
model.save_weights("./.weights/")

EPOCH : 0 | FILE : 0
Epoch 1/3
125/125 [==============================] - 52s 412ms/step - loss: 4.3200 - accuracy: 0.0738
Epoch 2/3
125/125 [==============================] - 50s 400ms/step - loss: 2.6148 - accuracy: 0.3324
Epoch 3/3
125/125 [==============================] - 47s 373ms/step - loss: 1.5074 - accuracy: 0.5800


KeyboardInterrupt: 